<a href="https://colab.research.google.com/github/tradeasystems/colab_test/blob/master/yahoo_pyfolio_portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install yfinance
!pip install pyfolio

#Restart environment

In [0]:
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive')

##Download portfolio from [yahoo](https://es.finance.yahoo.com/portfolios)

In [0]:
#I export yahoo portfolio into this folder in google drive
# my yahoo portfolio has in comments a structure to know exposure to different assets/countries by a dot
# Comment column
portfolio = pd.DataFrame.from_csv('/content/drive/My Drive/Colab Notebooks/quotes.csv')
portfolio.head()


## Update current price

In [0]:
import yfinance as yf

symbols = set(portfolio.index)
symbols_str = ' '.join(str(e) for e in symbols)
last_data = yf.download(symbols_str)
last_data.tail()

In [0]:
updated_current_price = []
for symbol in portfolio.index:
  updated_current_price.append(last_data['Close'][symbol][-1])
portfolio['Current Price'] = updated_current_price
portfolio.head()

In [0]:

money_invest = portfolio['Purchase Price']*portfolio['Quantity']

money_invest_total = money_invest.sum()
print('invested %d dollars'%money_invest_total)
money_invest

In [0]:
portfolio['open_pnl'] = (portfolio['Current Price']-portfolio['Purchase Price'])*portfolio['Quantity']
print('current pnl %d dollars'%portfolio['open_pnl'].sum())
portfolio['open_pnl']

### PnL



In [0]:
#returns
from datetime import datetime
close_historical = last_data['Close']
returns_historical = (last_data['Close']/last_data['Close'].shift(1))-1
start_portfolio = str(min(portfolio['Trade Date']))
start_portfolio
start_portfolio_date = datetime.strptime(start_portfolio, '%Y%m%d').date()
returns_historical = returns_historical[start_portfolio:].fillna(method='ffill')
returns_historical.head()

In [0]:
#position
position_historical = pd.DataFrame(0,index=returns_historical.index,columns = returns_historical.columns)
for index, row in portfolio.iterrows():
  date_str = str(row['Trade Date'])
  date = datetime.strptime(date_str, '%Y%m%d').date()
  symbol = str(index)
  position = row['Quantity']
  position_historical[symbol][date:] +=position 
position_historical.tail()

#### Historical Pnl


In [0]:
import matplotlib.pyplot as plt
figsize= (15,10)
portfolio_returns = position_historical*returns_historical
portfolio_returns['total'] = portfolio_returns.sum(axis=1)
open_pnl = portfolio_returns.cumsum()
legend=[]
for column in open_pnl.columns:
  if column is not 'total':
    legend.append(column)  
    open_pnl[column].plot(figsize=figsize)
  else:
    legend.append('open_pnl')  
    open_pnl[column].plot(figsize=figsize,color='purple',lw=3)

plt.legend(legend)
plt.grid()

sharpe = (portfolio_returns['total'].mean()/(portfolio_returns['total'].std()*252))

plt.title(' open_pnl=%.3f percent  sharpe=%.6f'%(open_pnl['total'][-1],sharpe))

In [0]:
portfolio_returns['total'].head()

#### PyFolio Returns


In [0]:
import pyfolio
pyfolio.create_returns_tear_sheet(portfolio_returns['total'],benchmark_rets=portfolio_returns['SPY'], return_fig=True)

##Portfolio exp
Remember that comments in yahoo need to have format
 %asset_type%.%country%

In [0]:
portfolio["asset_class"] = portfolio["Comment"]
portfolio["geography"] = portfolio["Comment"]
portfolio[['asset_class','geography']] = portfolio['Comment'].str.split('.',expand=True)
portfolio['position_pct'] = (portfolio['Current Price']*portfolio['Quantity'])/ (portfolio['Current Price']*portfolio['Quantity']).sum()
portfolio.head()

In [0]:
print(portfolio.groupby('asset_class').sum()['position_pct'])
portfolio.groupby('asset_class').sum()['position_pct'].plot.pie(y='position_pct', figsize=(7, 7))

In [0]:
print(portfolio.groupby('geography').sum()['position_pct'])
portfolio.groupby('geography').sum()['position_pct'].plot.pie(y='position_pct', figsize=(7, 7))